In [ ]:
# Install required packages
!pip install -q chromadb sentence-transformers

## Initialize Vector Store

In [ ]:
from traditional_rag_vectorstore import TraditionalRAGVectorStore
import json

# Initialize ChromaDB
vectorstore = TraditionalRAGVectorStore(
    collection_name="vietnamese_history_chunks",
    persist_directory="./chroma_db"
)

print(f"\nModel: {vectorstore.model_name}")
print(f"Embedding dimension: {vectorstore.embedding_dim}")
print(f"Current chunks: {vectorstore.count_chunks():,}")

## Load Chunks from JSON

In [ ]:
# Load chunks từ file có sẵn
with open('./data/chunk/chapter10_chunk.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

chunks = data['chunks']

print(f"=== CHUNK DATA ===")
print(f"Source file: {data['source_file']}")
print(f"Total chunks: {data['total_chunks']:,}")
print(f"Config: {data['config']}")

# Sample chunk
print(f"\n=== SAMPLE CHUNK ===")
sample = chunks[50]
print(f"ID: {sample['id']}")
print(f"Metadata: {sample['metadata']}")
print(f"Content: {sample['content'][:200]}...")

## Add Chunks to Vector Store

⚠️ **Chỉ chạy cell này một lần!** Nếu đã add rồi thì skip.

In [ ]:
# Add chunks to ChromaDB
stats = vectorstore.add_chunks(chunks, batch_size=32, show_progress=True)

print(f"\n=== ADD STATISTICS ===")
print(f"Total chunks: {stats['total']:,}")
print(f"Successfully added: {stats['success']:,}")
print(f"Failed: {stats['failed']}")

## Check Vector Store Stats

In [ ]:
# Get stats
stats = vectorstore.get_stats()

print("=== VECTOR STORE STATS ===")
for key, value in stats.items():
    print(f"{key}: {value}")

## Test Semantic Search

In [ ]:
# Test query
query = "Cách mạng tháng Tám năm 1945 thành công do đâu?"

print(f"Query: {query}")
print(f"{'='*80}\n")

results = vectorstore.search_similar(query, top_k=5)

for i, result in enumerate(results, 1):
    print(f"{i}. Chunk ID: {result['id']}")
    print(f"   Similarity: {result['similarity']:.4f}")
    print(f"   Pages: {result['metadata']['page_start']}-{result['metadata']['page_end']}")
    print(f"   Word count: {result['metadata']['word_count']}")
    print(f"   Content preview: {result['content'][:150]}...")
    print()

## Test Multiple Queries

In [ ]:
test_queries = [
    "Ai là người lãnh đạo phong trào cách mạng?",
    "Các chiến dịch quân sự quan trọng",
    "Cuộc kháng chiến chống Pháp",
    "Chính phủ Việt Nam dân chủ cộng hòa",
    "Mặt trận Việt Minh"
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    print(f"{'='*80}")
    
    results = vectorstore.search_similar(query, top_k=3)
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. {result['id']} (pages {result['metadata']['page_start']}-{result['metadata']['page_end']})")
        print(f"   Similarity: {result['similarity']:.4f}")
        print(f"   {result['content'][:120]}...")

## Test Retrieval by Page

In [ ]:
# Get all chunks from a specific page
page_number = 100

chunks_in_page = vectorstore.get_chunks_by_page(page_number)

print(f"Chunks in page {page_number}: {len(chunks_in_page)}")
print()

for chunk in chunks_in_page:
    print(f"ID: {chunk['id']}")
    print(f"Pages: {chunk['metadata']['page_start']}-{chunk['metadata']['page_end']}")
    print(f"Content: {chunk['content'][:100]}...")
    print()

## Compare with Graph RAG Query

So sánh kết quả Traditional RAG với Graph RAG cho cùng một query

In [ ]:
query = "Cách mạng tháng Tám năm 1945 thành công do đâu?"

print("=" * 80)
print("TRADITIONAL RAG (Vector-only)")
print("=" * 80)

trad_results = vectorstore.search_similar(query, top_k=5)

for i, result in enumerate(trad_results, 1):
    print(f"\n{i}. Similarity: {result['similarity']:.4f}")
    print(f"   Pages: {result['metadata']['page_start']}-{result['metadata']['page_end']}")
    print(f"   {result['content'][:120]}...")

print("\n" + "=" * 80)
print("Note: Graph RAG sẽ có thêm context từ relationships")
print("Traditional RAG chỉ dựa vào vector similarity")
print("=" * 80)

## Cleanup (Optional)

⚠️ Chỉ chạy nếu muốn xóa toàn bộ data và start lại từ đầu

In [ ]:
# Uncomment để xóa collection
# vectorstore.delete_all()
# print(f"Current chunks: {vectorstore.count_chunks()}")